# **Optimization of Term Reduction in Typeless $\lambda$-Calculus**

## Import/Define required modules and functions

In [1]:
class LambdaError(Exception):
    __errmsg = [
        "unrecognised error",
    ]

    def __init__(self, errDescription):
        if isinstance(errDescription, int):
            try:
                self._msg = LambdaError.__errmsg[errDescription]
            except:
                self._msg = LambdaError.__errmsg[0]
        elif isinstance(errDescription, str):
            self._msg = errDescription
        else:
            self._msg = LambdaError.__errmsg[0]
        super().__init__(self._msg)

## Syntax

### Variables

The set $\mathbf{Var}$ is the set of variables that are atomic entities of the typeless $\lambda$-calculus, each of that refers to itself only.
We assume the existence of an infinite enumerable series of variables.

So, we propose the following computational model to represent a variable.<br/>
This model assumes that a variable is a decorated natural number.
A variable is represented as '`#k`' ($k\in\mathbb N$).

In [2]:
class Var:
    __cvar = 0

    def __init__(self):
        self._data = Var.__cvar
        Var.__cvar += 1

    def __str__(self):
        return f"#{self._data}"

    def __eq__(self, another):
        if isinstance(another, Var):
            return self._data == another._data
        raise LambdaError("Var.__eq__ waits for an instance of Var"
                          f", but it received '{another}'")

Some examples of variables.

In [3]:
# x, y, z = Var(), Var(), Var()
#
# print(f"x = {x}\ny = {y}\nz = {z}")

### Terms

The set $\mathbf{Term}$ of $\lambda$-terms (or briefly terms) is defined by the following rules.

---

$$\tag{$\Lambda$1}
\begin{equation}
\dfrac{x:\mathbf{Var}}{x:\mathbf{Term}}
\end{equation}
$$

$$
\tag{$\Lambda$2}
\begin{equation}
\dfrac{t_1:\mathbf{Term}\qquad t_2:\mathbf{Term}}{(t_1\ t_2):\mathbf{Term}}
\end{equation}
$$

$$\tag{$\Lambda$3}
\begin{equation}
\dfrac{x:\mathbf{Var}\qquad t:\mathbf{Term}}{(\operatorname{\lambda}x\mathop.t):\mathbf{Term}}
\end{equation}
$$

---

One usually uses the following rules for omitting parentheses

1. the outer parentheses omit always;
1. the term of the form $((t_1\ t_2)\ t_3)$ abbreviates to $t_1\ t_2\ t_3$;
1. the term of the form $(\operatorname{\lambda}x_1\mathop.(\operatorname{\lambda}x_2\mathop.t))$ abbreviates to $\operatorname{\lambda}x_1\mathop.\operatorname{\lambda}x_2\mathop.t$;
1. the term of the form $(\operatorname{\lambda}x\mathop.(t_1\ t_2))$ abbreviates to $\operatorname{\lambda}x\mathop.t_1\ t_2$.

The following classes represent $\lambda$-terms.<br/>
The class represents
* the atomic term $x$ where $x=\mathtt{\#}k$ ($ k\in\mathbb N$) as '$\mathtt{\$}k$'
* the application term $(t_1\ t_2)$ where $t_1$ and $t_2$ are terms as $\mathtt(t_1\ \mathtt.\ t_2\mathtt)$
* the abstraction term $\operatorname{\lambda}x\mathop.t$ where $x$ is a variable $\mathtt{\#}k$ and $t$ is a term as $\mathtt{(@}k\mathtt{ . }t\mathtt)$


In [30]:
class Term:  # the basic abstract class for representing a term

    @property
    def kind(self):  # returns the kind of the term
        if isinstance(self, Atom):
            return "atom"
        if isinstance(self, Application):
            return "application"
        if isinstance(self, Abstraction):
            return "abstraction"

    def __str__(self):
        if self.kind == "atom":
            return f"{self._data}"
        if self.kind == "application":
            return f"({self._data[0]} {self._data[1]})"
        else:  # self.kind == "absraction"
            return f"(λ{self._data[0]}. {self._data[1]})"

    # def __eq__(self, another):
    #     if isinstance(another, Term):
    #         if self.kind != another.kind:
    #             return False
    #         return self._data == another._data
    #     else:
    #         raise LambdaError(3)
    def __eq__(self, other):
        if not isinstance(other, Term):
            return False
        if self.kind == "atom" and other.kind == "atom":
            return self._data == other._data
        if self.kind == "application" and other.kind == "application":
            return self._data[0] == other._data[0] and self._data[1] == other._data[1]
        if self.kind == "abstraction" and other.kind == "abstraction":
            return self._data[0] == other._data[0] and self._data[1] == other._data[1]
        return False

    def call_as_method(self, fun, *args):
        return fun(self, *args)

    @property
    def is_beta_redex(self):
        """:return: bool is the term is a beta-redex"""
        return (self.kind == "application") and (self._data[0].kind == "abstraction")

    @property
    def redexes(self):
        """:return: list of all beta-redexes in the term"""
        if self.kind == "atom":
            return []
        if self.kind == "abstraction":
            return self._data[1].redexes
        # self is App:
        redexes_list = [self] if self.is_beta_redex else []
        redexes_list += self._data[0].redexes + self._data[1].redexes
        return redexes_list

    @property
    def _vars(self):
        """
        Here, keys of the external dictionary are the variables that
        are occurred in 'self', and values of the internal dictionaries
        relate respectively to the numbers of free and bound occurrences
        of the variables.
        :return: dict[Var, dict[('free'/'bound'), int]]
        """
        if self.kind == "atom":
            return {self._data: {"free": 1, "bound": 0}}
        if self.kind == "application":
            vars_, auxvars_ = dict(self._data[0]._vars), self._data[1]._vars
            for var_ in auxvars_:
                try:
                    for key_ in ("free", "bound"):
                        vars_[var_][key_] += self._data[1]._vars[var_][key_]
                except KeyError:
                    vars_[var_] = dict(self._data[1]._vars[var_])
            return vars_
        # self is Abstraction:
        vars_ = dict(self._data[1]._vars)
        try:
            vars_[self._data[0]]["bound"] += vars_[self._data[0]]["free"]
            vars_[self._data[0]]["free"] = 0
        except KeyError:
            pass
        return vars_

    @property
    def vertices_number(self):
        """:return: the number of nodes in the tree representation the lambda term"""
        if self.kind == "atom":
            return 1
        if self.kind == "application":
            return 1 + self._data[0].vertices_number + self._data[1].vertices_number
        # self is Abstraction
        return 1 + self._data[1].vertices_number

    def normalize(self, strategy):
        """
        :param strategy: OneStepStrategy
        :return: tuple of the normal form of the term and number of steps of betta reduction
        """
        term = self._update_bound_vars()
        count_steps = 0
        while len(term.redexes) > 0:
            term = term._beta_conversion(strategy)._update_bound_vars()
            count_steps += 1
            # computation limitation
            if (term.vertices_number > 7_000) or (count_steps > 400):
                return term, float("inf")
        return term, count_steps

    def _beta_conversion(self, strategy):
        """
        :param strategy: OneStepStrategy
        :return: term with redex eliminated using the given strategy
        """
        index = strategy.redex_index(self)
        subterm_ = self.subterm(index)
        reduced_term = subterm_._remove_outer_redex()
        return self.set_subterm(index, reduced_term)

    def subterm(self, index: int):
        """
        By representing the term as a tree, a subtree is returned,
        which is also a lambda term.
        The vertex of this subtree has a given index in the topological
        sorting of the vertices of the original term.
        :param index: int subterm index
        :return: subterm: Term
        """
        if index == 1:
            return self
        if self.kind == "atom":
            raise ValueError("index value is incorrect")
        elif self.kind == "application":
            if self._data[0].vertices_number + 1 >= index:
                return self._data[0].subterm(index - 1)
            else:
                return self._data[1].subterm(index - self._data[0].vertices_number - 1)
        else:
            return self._data[1].subterm(index - 1)

    def set_subterm(self, index: int, term):
        """
        By representing the term as a tree, a subtree is set, which is also a lambda term.
        The vertex of this subtree has a given index in the topological sorting of the vertices of the original term.
        :param index: subterm index
        :param term: λ-term to which the subterm will be replaced
        :return: updated λ-term
        """
        if index == 1:
            return  term

        if self.kind == "atom":
            raise ValueError("index value is incorrect")
        elif self.kind == "application":
            if self._data[0].vertices_number + 1 >= index:
                return Application(self._data[0].set_subterm(index - 1, term), self._data[1])
            else:
                return Application(self._data[0],
                                   self._data[1].set_subterm(index - self._data[0].vertices_number - 1, term))
        else:
            return Abstraction(self._data[0], self._data[1].set_subterm(index - 1, term))

    def _update_bound_vars(self):
        """:return: λ-term with updated bound variables"""
        if self.kind == "atom":
            return self
        if self.kind == "application":
            return Application(
                self._data[0]._update_bound_vars(),
                self._data[1]._update_bound_vars()
            )
        # self is abstraction
        new_var = Var()
        return Abstraction(
            new_var,
            self._data[1]._replace_variable(self._data[0], Atom(new_var))._update_bound_vars()
        )

    def _remove_outer_redex(self):
        """Apply the betta conversion to the lambda term, removing the outer betta redex"""
        if self.is_beta_redex:
            head = self._data[0]._data[0]
            body = self._data[0]._data[1]
            return body._replace_variable(head, self._data[1])
        else:
            return self

    def _replace_variable(self, var: Var, term):
        """Return λ-term with replaced variable"""
        if self.kind == "atom":
            return term if self._data == var else self
        if self.kind == "application":
            return Application(self._data[0]._replace_variable(var, term),
                               self._data[1]._replace_variable(var, term))
        # self is abstraction
        return Abstraction(self._data[0], self._data[1]._replace_variable(var, term))



class Atom(Term):  # the class of terms created with the first rule

    def __init__(self, v):
        if isinstance(v, Var):
            self._data = v
        else:
            raise LambdaError("Atom.__init__ waits for an instance of Var"
                              f", but it received '{v}'")


class Application(Term):  # the class of terms created with the second rule

    def __init__(self, t1, t2):
        if isinstance(t1, Term) and isinstance(t2, Term):
            self._data = (t1, t2)
        else:
            raise LambdaError("Application.__init__ waits for two instances"
                              f" of Term, but it received '{t1}', '{t2}'")


class Abstraction(Term):  # the class of terms created with the third rule

    def __init__(self, v, t):
        if isinstance(v, Var) and isinstance(t, Term):
            self._data = (v, t)
        else:
            raise LambdaError("Abstraction.__init__ waits for an instance of"
                              " Var and an instance of Term"
                              f", but it receive '{v}' and '{t}'")

In [31]:
from abc import ABC, abstractmethod


class OneStepStrategy(ABC):
    @abstractmethod
    def redex_index(self, term: Term, init_index=0) -> int:
        """
        :return: index of the vertex of a subterm that has an outer redex.
                The index of a vertex is the index of this vertex in the topological sort of the tree vertices.
                Indexing starts at 1.
        """


class LOStrategy(OneStepStrategy):
    def redex_index(self, term: Term, init_index=0) -> int:
        if (term.kind == "atom") or (len(term.redexes) == 0):
            raise ValueError("The term doesn't contain a redex")
        if term.kind == "application":
            if term.is_beta_redex:
                return init_index + 1
            if len(term._data[0].redexes) != 0:
                return self.redex_index(term._data[0], init_index + 1)
            return self.redex_index(term._data[1],
                                    init_index + term._data[0].vertices_number + 1)
        # self is Abstraction:
        return self.redex_index(term._data[1], init_index + 1)

In [32]:
Lambda = Abstraction
App = Application

In [33]:
x, y = Var(), Var()
x_, y_ = Atom(x), Atom(y)

test_term = App(
    Lambda(x, Lambda(y, y_)),
    App(
        Lambda(x, App(x_, x_)),
        Lambda(x, App(x_, x_))
    )
)

print(test_term)

((λ#6447. (λ#6448. #6448)) ((λ#6447. (#6447 #6447)) (λ#6447. (#6447 #6447))))


In [34]:
norm_term, count = test_term.normalize(LOStrategy())
print(count)
print(norm_term)

1
(λ#6453. #6453)


Some examples of terms

In [ ]:
tx, ty, tz = Atom(x), Atom(y), Atom(z)
# λx. x
tI = Abstraction(x, tx)
# λxy. x
tK = Abstraction(x, Abstraction(y, tx))
# λxyz. (x z) (y z)
tS = Abstraction(
    x,
    Abstraction(
        y,
        Abstraction(
            z,
            Application(
                Application(tx, tz),
                Application(ty, tz)))))

print(f"x = {tx}")
print(f"I = {tI}")
print(f"K = {tK}")
print(f"S = {tS}")

### Paths

We use the concept of a ***path***.

A path is syntactically a string of '$\mathtt l$', '$\mathtt d$', and '$\mathtt r$'.
The set of paths is referred to as $\Pi$.

In [ ]:
def is_path(s):
    return isinstance(s, str) and len(s) == len([c for c in s if c in "ldr"])

A path is used for referring to a subterm of a term using the partially defined function $\operatorname{subref}:\mathbf{Term}\times\Pi\dashrightarrow\mathbf{Term}$.

$$\begin{array}{lll}
    \operatorname{subref}\ t\ \epsilon&=t&\textsf{for any term }t \\
    \operatorname{subref}\ (t_1\,t_2)\ \mathtt l\cdot\pi&=t_1&\textsf{for any terms }t_1,\ t_2\textsf{ and path }\pi \\
    \operatorname{subref}\ (t_1\,t_2)\ \mathtt r\cdot\pi&=t_2&\textsf{for any terms }t_1,\ t_2\textsf{ and path }\pi \\
    \operatorname{subref}\ (\lambda\,x\mathop{.}t)\ \mathtt d\cdot\pi&=t&\textsf{for any variable }x,\textsf{ term }t,\textsf{ and path }\pi \\
    \operatorname{subref}\ t\ \pi&\textsf{ is undefined }&\textsf{for all other cases}
\end{array}$$

The program realisation of this function is `subref(t: Term, p: Path) -> Term | None` specified here.<br/>
It returns the corresponding subterm or None if this subterm is undefined.

In [ ]:
def subref(t, p):
    if isinstance(t, Term) and is_path(p):
        if p == "":
            return t
        if p[0] == 'l' and t.kind == "application":
            return subref(t._data[0], p[1:])
        if p[0] == 'r' and t.kind == "application":
            return subref(t._data[1], p[1:])
        if p[0] == 'd' and t.kind == "abstraction":
            return subref(t._data[1], p[1:])
        # all other cases
        return None
    raise LambdaError("'subref' waits for an instance of Term and a path"
                      f", but it received '{t}' and '{p}'")

The set of paths for a term $t$ is defined as follows
$$\operatorname{\Pi}(t)=\{\pi\in\Pi\mid\operatorname{subref}\ t\ \pi\textsf{ is defined}\}.$$

In [ ]:
def paths(t):
    """collects all paths that refer to some correct subterm of 't'
    Result is a dictionary whose keys are paths determining
        the corresponding subterm
    """
    if isinstance(t, Term):
        result = {"": t}
        if t.kind == "atom":
            return result
        if t.kind == "application":
            return {**result,
                    **{("l" + key): val for (key, val) in
                       paths(subref(t, "l")).items()},
                    **{("r" + key): val for (key, val) in
                       paths(subref(t, "r")).items()}}
        # t.kind == "abstraction"
        return {**result,
                **{("d" + key): val for (key, val) in
                   paths(subref(t, "d")).items()}}
    raise LambdaError("'paths' waits for an instance of Term"
                      f", but it received '{t}'")

For example, the next cell computes the corresponding dictionary for combinator `tS`.

In [ ]:
pths = tS.call_as_method(paths)
for key in pths.keys():
    print(f"'{key}': {pths[key]}")

This example illustrates the following fact.

**Proposition.**
For any term $t$, $\Pi(t)$ is a prefix closed finite subset of $\Pi(t)$.

In some sense, $\Pi(t)$ is the "skeleton" of $t$.
Terms with the same skeleton are similar.<br/>
This leads us to the function `similar`.

In [ ]:
def similar(t1, t2):
    if isinstance(t1, Term) and isinstance(t2, Term):
        return paths(t1).keys() == paths(t2).keys()
    raise LambdaError("'similar' waits for two instances of Term"
                      f", but it received '{t1}' and '{t2}'")

Let us consider the next examples.

In [ ]:
t1 = Application(tx, tI)
another_tI = Abstraction(y, ty)
t2 = Application(ty, another_tI)
print(f"{tI} and {another_tI}"
      f" are {'' if similar(tI, another_tI) else 'not '}similar")
print(f"{t1} and {t2} are {'' if similar(tI, another_tI) else 'not '}similar")

In [ ]:
tN = Abstraction(y, tx)
print(f"{tI} and {tN}"
      f" are {'' if similar(tI, another_tI) else 'not '}similar")

Maximal paths in $\Pi(t)$ lead to variables.
There are two kinds of these paths:
* such ones that do not have a prefix, which refers to the abstraction-subterm with the variable equal to the variable, to which this path refers;
* and such ones that have a prefix, which refers to the abstraction-subterm with the variable equal to the variable, to which this path refers.

Paths of the first kind refer to free variables, and ones of the second kind refer to bound variables.

In [ ]:
def vars(t):
    """builds a dictionary, in which keys are refs to term variables,
    values are pairs constructed from the corresponding variable and
    the ref to the abstraction-superterm that bound the variable if
    it is bound or None elsewhen.
    """
    varoccs = {key: st._data
               for (key, st) in paths(t).items() if st.kind == "atom"}
    result = {}
    for key in varoccs:
        free = True
        for ie in range(1, len(key) + 1):
            subkey = key[: - ie]
            term = subref(t, subkey)
            if (term.kind == "abstraction" and
                    term._data[0] == varoccs[key]):
                result[key] = (varoccs[key], subkey)
                free = False
                break
        if free:
            result[key] = (varoccs[key], None)
    return result

In [ ]:
print(tS)
for key in vars(tS):
    print(f"'{key}': {vars(tS)[key]}")

In [ ]:
lst = [0, 1, 2, 3, 4]
for i in range(1, len(lst) + 1):
    print(lst[:- i])

In [ ]:
len("stuf;th;ju;".split(';'))

(λ0. (λ1. (λ2. ((0 2) (1 2)))))
'dddll': (0, '')
'dddlr': (2, 'dd')
'dddrl': (1, 'd')
'dddrr': (2, 'dd')